In [ ]:
# Set up

# docker pull qdrant/qdrant

# docker run -p 6333:6333 -p 6334:6334 \
#    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
#    qdrant/qdrant

!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [1]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%pip install fastembed

In [2]:
import numpy as np
from fastembed import TextEmbedding

### Q1. Embedding the query

In [26]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
text = "I just discovered the course. Can I join now?"

q = list(model.embed(text))[0]

In [27]:
# check if it is already normalized
np.linalg.norm(q)

np.float64(1.0)

In [28]:
# Q: What's the minimal value in this (vector) array?

np.min(q)

np.float64(-0.11726373885183883)

### Q2. Cosine similarity with another vector

In [30]:
# compute the cosine of the query vector with itself, the result will be 1.0
q.dot(q)

np.float64(1.0000000000000002)

In [31]:
doc = 'Can I still join the course after the start date?'

doc_vector = list(model.embed(doc))[0]

In [32]:
# Q: What's the cosine similarity between the vector for the query and the vector for the document?

q.dot(doc_vector)

np.float64(0.9008528895674548)

### Q3. Ranking by cosine

In [34]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
docs = []
for doc in documents:
    docs.append(doc['text'])

V = np.array(list(model.embed(docs)))

In [48]:
V.dot(q)

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [ ]:
# Q: What's the document index with the highest similarity? (Indexing starts from 0)?

# A: 1

### Q4. Ranking by cosine, version two

In [49]:
docs = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    docs.append(full_text)

V = np.array(list(model.embed(docs)))

In [50]:
V.dot(q)

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [ ]:
# Q: Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

# A: 0

### Q5. Selecting the embedding model

In [52]:
min_dim = float('inf')

for model in TextEmbedding.list_supported_models():
    min_dim = min(min_dim, model["dim"])
    
print(f"The smallest dimensionality for models in fastembed is {min_dim}.")

The smallest dimensionality for models in fastembed is 384.


In [53]:
import json

for model in TextEmbedding.list_supported_models():
    if model["dim"] == min_dim:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [ ]:
# Q: What's the smallest dimensionality for models in fastembed?

# A: 384

In [59]:
model_handle = "BAAI/bge-small-en"

### Q6. Indexing with qdrant

In [54]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [55]:
# Define the collection name
collection_name = "zoomcamp-qa"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_dim,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [60]:
points = []
id = 0

for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1
    
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  6.62it/s]


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )
    return results

In [62]:
query = "I just discovered the course. Can I join now?"

result = search(query)
result

QueryResponse(points=[ScoredPoint(id=14, version=1, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [ ]:
# Q: What's the highest score in the results? (The score for the first returned record)?

# A: 0.8703